# Test 2

> Test Fast.ai models with a simple supervised learning test

In [ ]:
#| default_exp test2

In [ ]:
#| hide
from nbdev.showdoc import *

%matplotlib inline
from fastai.vision.all import *
from fastai.data.all import *
import torch.nn.functional as F

from Noise2Model.core import *

from IPython.display import clear_output, DisplayHandle
def update_patch(self, obj):
    clear_output(wait=True)
    self.display(obj)
DisplayHandle.update = update_patch

In [ ]:
bs, size = 4, 512
# arch = models.resnet34
path = Path('../_data/Confocal_BPAE_B/')
path_train_raw = Path(path/"raw")
path_train_gt = Path(path/"gt")

In [ ]:
dblock = DataBlock(blocks    = (ImageBlock(cls=PILImageBW), ImageBlock(cls=PILImageBW)),
                   get_items = get_image_files, 
                   get_y = get_noisy_pair,
                   splitter  = RandomSplitter(valid_pct=0.2),
                   item_tfms = RandomCrop(64),
                   )
# dblock.summary(path_train_raw)
# dsets = dblock.datasets(path_train_raw)
# dsets.train[0]
dls = dblock.dataloaders(path_train_raw)
dls.show_batch(max_n=8, cmap='gray')

In [ ]:
# m = create_vision_model(models.resnet18, 1, False, n_in=1)
# print(m.children)


In [ ]:

# m = nn.Sequential(*list(m.children())[:-1])
# #m.children

# mm = DynamicUnet(m, 1, (64,64), norm_type=None)
#mm.children

In [ ]:
# learn = Learner(dls, mm, loss_func = F.l1_loss)
# learn.summary()

In [ ]:
# learn.lr_find()

In [ ]:
# learn.fit_one_cycle(20, lr_max = 1e-04)

In [ ]:
# learn.show_results(max_n=1, cmap='gray')

In [ ]:
learn = unet_learner(dls, models.resnet18, loss_func=F.l1_loss, n_in=1, n_out=1, pretrained=False, cut=None)
learn.summary()

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(20, lr_max = 1e-04)

In [ ]:
learn.show_results(max_n=10, cmap='gray')

In [13]:
#| hide
import nbdev; nbdev.nbdev_export()